In [1]:
import numpy as np
from hmmlearn import hmm
import csv


In [2]:
hmm_states = 2

model = hmm.CategoricalHMM(n_components=hmm_states)
model.emissionprob_ = np.array([[0.4,0.6],[0.3,0.7]])
model.transmat_ = np.array([[0.2,0.8],[0.1,0.9]])
model.startprob_ = np.array([0.4,0.6])
data,states = model.sample(n_samples = 1000,random_state=28)
data = np.squeeze(data)

In [3]:

# constant n_obs
# n_obs = 1000
# # Set p(H) apriori for simulation as a biased coin.
# p_h = 0.8

# # Model experiment as a single biased coin flipped 1000 times.
# data = np.random.binomial(1, p_h, n_obs=n_obs)

In [4]:
n_states = 2
n_features = 2
n_obs = data.shape[0]
# emission = np.random.rand(n_obs,n_states)
# emission = emission/np.tile(np.expand_dims(np.sum(emission,axis=1),axis=1),(1,2))

emission = np.random.rand(n_states,n_features)
emission = emission/np.tile(np.expand_dims(np.sum(emission,axis=1),axis=1),(1,2))
# emission = np.array([[0.5,0.5],[0.5,0.5]])

transition =  np.random.rand(n_states,n_states)
transition = transition/np.tile(np.expand_dims(np.sum(transition,axis=1),axis=1),(1,2))
# transition = np.array([[0.6,0.4],[0.5,0.5]])


# forward = np.random.rand(n_obs,n_states)
scale_factors = np.zeros((n_obs))
forward_hat = np.zeros((n_obs,n_states))

# backward = np.random.rand(n_obs,n_states)
backward_hat = np.zeros((n_obs,n_states))

init_prob = np.array([0.5,0.5])


p_old = -10000
tol = 0.01
max_iter = 100

mu = np.random.rand(n_states)

In [5]:
print(f'emission is {emission} and transition is {transition}')

emission is [[0.48392847 0.51607153]
 [0.94652593 0.05347407]] and transition is [[0.47933558 0.52066442]
 [0.43619813 0.56380187]]


In [6]:
for ite in range(max_iter):
    forward_hat = np.zeros((n_obs,n_states))
    backward_hat = np.zeros((n_obs,n_states))
    scale_factors = np.zeros((n_obs))

    for i in range(n_states):
        forward_hat[0,i] = init_prob[i] * emission[i,data[0]]
    scale_factors[0] = np.sum(forward_hat[0,:])
    forward_hat[0,:] = forward_hat[0,:]/scale_factors[0]
    
    for t in range(1,n_obs):
        for i in range(n_states):
            for j in range(n_states):
                forward_hat[t,i] += forward_hat[t-1,j] * transition[j,i]
                # temp = np.matmul(forward_hat[t,i] ,np.transpose(transition[i,j])) * emission[i,data[t+1]]
            forward_hat[t,i] = forward_hat[t,i] * emission[i,data[t]]
        scale_factors[t] = np.sum(forward_hat[t,:])

        forward_hat[t,:] = forward_hat[t,:]/scale_factors[t]
        # print(f'temp is {temp} and the scale factor is {scale_factors[t+1]} and the forward_hat is {forward_hat[t+1]}')

        # forward[t,i] = forward_hat[t,:]*np.prod(scale_factors[0:t])

    backward_hat[-1,:] = scale_factors[-1]
    for t in reversed(range(n_obs-1)):
        for i in range(n_states):
            for j in range(n_states):
                backward_hat[t,i] += backward_hat[t+1,j] * emission[j,data[t+1]] * transition[i,j]
        backward_hat[t,:] = backward_hat[t,:]/scale_factors[t]


    a = np.zeros((n_obs,n_states))
    b = np.zeros((n_obs,n_states,n_states))
    for i in range(n_obs):
        for j in range(n_states):
            a[i,j] = forward_hat[i,j]*backward_hat[i,j]
        temp = np.sum(a[i,:])
        a[i,:] = a[i,:]/temp
    for t in range(n_obs-1):
        for i in range(n_states):
            for j in range(n_states):
                b[t,i,j] = scale_factors[t+1]*forward_hat[t,i]*backward_hat[t+1,j]*transition[i,j]*emission[j,data[t+1]]

    for i in range(n_states):
        for j in range(n_states):
            transition[i,j] = np.sum(b[0:-1,i,j])/np.sum(b[0:-1,i,:])
            # print(np.sum(b[0:-1,i,j]),np.sum(b[0:-1,i,:]))


    init_prob = a[0,:]

    for i in range(n_states):
        for j in range(n_states):
            emission[j,i] = np.sum(a[np.argwhere(data==i),j]) / np.sum(a[:,j])
            
    p = np.sum(np.log(scale_factors))
    # print(f'p is:{p},transition is {transition},emission is {emission},init is {init_prob}')
    print(f'p is: {p}')
    # print(f'transition prob is: {transition}')
    # print(f'emission prob is:{emission}')
    if p>p_old and p - p_old < tol:
        break
    p_old = p

p is: -998.4918484902878
p is: -638.5302220805
p is: -624.2442218326438
p is: -630.900873109427
p is: -632.1213967987817
p is: -629.0405237669353
p is: -625.2973067536612
p is: -623.3015535659624
p is: -622.5952982189926
p is: -622.3713974104254
p is: -622.2985412641015
p is: -622.2742139379113
p is: -622.2662253520143


In [7]:
print(f'transition is {transition}, emission is {emission},init_prob is {init_prob}')

transition is [[0.99719223 0.00280777]
 [0.99746015 0.00253985]], emission is [[0.31134945 0.68865055]
 [0.89505096 0.10494904]],init_prob is [9.99999999e-01 6.50998438e-10]


In [9]:
# model = hmm.CategoricalHMM(n_components=n_states,emissionprob_prior=emission,transmat_prior=transition,startprob_prior=init_prob)
# model2 = hmm.CategoricalHMM(n_components=hmm_states,n_iter=1000)


In [10]:
# model2.fit(data.reshape(-1,1))
# hidden_states = model2.predict(data.reshape(-1,1))
# print("Most likely hidden states:", hidden_states)


In [11]:
# model2.monitor_.converged


In [12]:
# model2.emissionprob_

In [13]:
# model2.transmat_